### Authentication
First of all we authenticate in ee

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1ARtbsJowXNW0ndutkqzPCXAa9zbySO9oq5kfGyD00iBzkL4U70Fxmsa0Pm0

Successfully saved authorization token.


In [567]:
# Get the desired zone shapefile
import json
with open('/home/aarja/Downloads/cuadrado.geojson', 'r') as jf:
    fc = json.load(jf)
    totoral = ee.FeatureCollection(fc)

In [568]:
def clipper(image):
    global totoral
    return image.clip(totoral)

In [569]:
# Import the Sentinel-2 collection
#sentinel_2_msi = ee.ImageCollection("COPERNICUS/S2_SR").filterDate('2021-09-01', '2022-10-17').map(clipper)
sentinel_2_msi = ee.ImageCollection("COPERNICUS/S2").filterDate('2021-09-01', '2022-10-17').map(clipper)

In [570]:
def calculate_ndvi(image):
    ndvi = image.addBands(image.normalizedDifference(['B8', 'B4']))
    return ndvi

def calculate_evi(image):
    time = image.get('system:time_start')
    evi = image.expression(
        '2.5 * ((band_nir - band_red) / ((band_nir + (6 * band_red) - (7.5 * band_blue)) + 1))',
        {
            'band_nir': image.select('B8'),
            'band_red': image.select('B4'),
            'band_blue': image.select('B2')
        }
    ).rename('evi').set('system:time_start', time)
    return evi    

In [571]:
sentinel_2_msi = sentinel_2_msi.map(calculate_ndvi)

In [572]:
import folium

In [573]:
lat, lon = -30.749311, -64.057867

my_map = folium.Map(location=[lat, lon], zoom_start=22)

In [574]:
# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))


# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [167]:
# Set visualization parameters for land cover.
vis_params = {
    'min': -1,'max': 1,
    'pallette': ['Red', 'Green']
}

vis_params = { 'min': 0.0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],}
SentinalNdvi = sentinel_2_msi.select('nd')
my_map.add_ee_layer(SentinalNdvi, vis_params, 'NDVI')

In [168]:
#my_map.add_ee_layer(totoral, {'color': 1, 'width': 3}, 'Totoral')
#my_map.add_child(folium.LayerControl())
#display(my_map)

In [575]:
evis = sentinel_2_msi.map(calculate_evi)
# Set visualization parameters for land cover.
vis_params = {
    'min': -1,'max': 1,
    'pallette': ['Red', 'Green']
}

vis_params = { 'min': 0.0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],}


In [799]:
sentinel_2_msi = ee.ImageCollection("COPERNICUS/S2_SR").filterDate('2022-07-01', '2022-07-5').map(clipper)

evis = sentinel_2_msi.map(calculate_evi)
# Set visualization parameters for land cover.
vis_params = {
    'min': 0,'max': 1,
    #'palette': ['Green', 'LightGreen', 'Blue','LightBlue', 'Yellow', 'LightYellow', 'Purple', 'Orange', 'Pink', 'Red']
    #'palette': ['Green', 'LightGreen', 'LightYellow', 'Yellow', 'Orange', 'Red']
    'palette': ['Red', 'Yellow', 'Green']
    #'palette': ["FFFF00", "FF0000", "008000" ]
}
"""
vis_params = { 'min': -1.0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],}
"""
lat, lon = -30.749311, -64.057867

my_map = folium.Map(location=[lat, lon], zoom_start=22)
my_map.add_ee_layer(evis, vis_params, 'evis')
display(my_map)

In [770]:
# Get the desired zone shapefile
import json
with open('/home/aarja/Downloads/totoral.geojson', 'r') as jf:
    fc = json.load(jf)
    totoral = ee.FeatureCollection(fc)
totoral = ee.Projection.transform('32720')
totoral = ee.FeatureCollection('users/adelarja/agro/totoral')

In [771]:
def clipper(image, shape=None):
    """Clip the image with the shape object."""
    global totoral
    shape = totoral if not shape else shape
    return image.clip(shape)

In [772]:
sentinel_2_msi = ee.ImageCollection("COPERNICUS/S2").filterDate('2022-10-01', '2022-10-17').map(clipper)

In [773]:
import folium

lat, lon = -30.749311, -64.057867

my_map = folium.Map(location=[lat, lon], zoom_start=22)

# Define a method for displaying Earth Engine image tiles on a folium map.
def add_ee_layer(self, ee_object, vis_params, name):
    
    try:    
        # display ee.Image()
        if isinstance(ee_object, ee.image.Image):    
            map_id_dict = ee.Image(ee_object).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.ImageCollection()
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):    
            ee_object_new = ee_object.mosaic()
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
            ).add_to(self)

        # display ee.Geometry()
        elif isinstance(ee_object, ee.geometry.Geometry):    
            folium.GeoJson(
            data = ee_object.getInfo(),
            name = name,
            overlay = True,
            control = True
        ).add_to(self)

        # display ee.FeatureCollection()
        elif isinstance(ee_object, ee.featurecollection.FeatureCollection):  
            ee_object_new = ee.Image().paint(ee_object, 0, 2)
            map_id_dict = ee.Image(ee_object_new).getMapId(vis_params)
            folium.raster_layers.TileLayer(
            tiles = map_id_dict['tile_fetcher'].url_format,
            attr = 'Google Earth Engine',
            name = name,
            overlay = True,
            control = True
        ).add_to(self)
    
    except:
        print("Could not display {}".format(name))


# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [774]:
evis = sentinel_2_msi.map(calculate_ndvi)
SentinalNdvi = evis.select('nd')
# Set visualization parameters for land cover.
vis_params = {
    'min': -1,'max': 1,
    #'palette': ['Green', 'LightGreen', 'Blue','LightBlue', 'Yellow', 'LightYellow', 'Purple', 'Orange', 'Pink', 'Red']
    #'palette': ['Green', 'LightGreen', 'LightYellow', 'Yellow', 'Orange', 'Red']
    'palette': ["008000", "FFFF00", "FF0000"]
}
lat, lon = -30.749311, -64.057867

my_map = folium.Map(location=[lat, lon], zoom_start=22)

my_map.add_ee_layer(SentinalNdvi.first(), vis_params, 'RGB Totoral')
my_map.add_ee_layer(totoral, {}, 'bsas')
my_map.add_child(folium.LayerControl())
display(my_map)

In [758]:
totoral.getInfo()

{'type': 'FeatureCollection',
 'columns': {'AREA': 'Float',
  'BORDER_COL': 'String',
  'BORDER_STY': 'String',
  'BORDER_WID': 'Integer',
  'CLOSED': 'String',
  'FILL_STYLE': 'String',
  'LAYER': 'String',
  'LIM_PROV_': 'Long',
  'LIM_PROV_I': 'Long',
  'NOMBRE': 'String',
  'PERIMETER': 'Float',
  'PROV_COD': 'Long',
  'system:index': 'String'},
 'version': 1665929559135537,
 'id': 'users/adelarja/Hidro2022/Buenos_Aires',
 'properties': {'system:asset_size': 80167},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-63.395191323689815, -34.382531853772356],
      [-63.394531321557736, -34.996558992614204],
      [-63.394531321557736, -34.99836050563486],
      [-63.39165072936987, -35.50035493320077],
      [-63.39021044696977, -35.75135211119832],
      [-63.38877017370221, -36.00234938090249],
      [-63.387163812350835, -36.25204345551671],
      [-63.38555737239031, -36.501737469415296],
      [-63.383951033777045, -36.751431534422075],


In [768]:
totoral.getInfo()

EEException: Projection: The CRS of a map projection could not be parsed.